# ANC StateGraph (Ollama Mode)

This notebook is Ollama-specific and uses the shared stategraph module.

- Provider: `ollama`
- Shared orchestration module: `naturalist_companion.stategraph_shared`


In [5]:
# Uncomment in fresh environments:
# %pip install -q -r ../requirements-ollama-dev.txt

from IPython.display import Image, Markdown, display

from naturalist_companion.stategraph_shared import (
    build_stategraph_app,
    run_i81_eval_harness,
    run_stategraph,
)


In [6]:
provider = 'ollama'
app = build_stategraph_app(provider=provider)
print('Compiled StateGraph successfully for provider:', provider)

# Render a real image (PNG bytes) instead of plain Mermaid text.
try:
    png_bytes = app.get_graph().draw_mermaid_png()
    display(Image(data=png_bytes))
except Exception as exc:
    display(Markdown(f'Graph render fallback (text). Error: `{type(exc).__name__}: {exc}`'))
    print(app.get_graph().draw_mermaid())


Compiled StateGraph successfully for provider: ollama
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	init_run(init_run)
	route_query(route_query)
	ask_clarification(ask_clarification)
	rewrite_query(rewrite_query)
	retrieve_parallel(retrieve_parallel)
	rerank_results(rerank_results)
	generate_structured_answer(generate_structured_answer)
	quality_gate(quality_gate)
	safe_stop(safe_stop)
	finalize(finalize)
	__end__([<p>__end__</p>]):::last
	__start__ --> init_run;
	ask_clarification --> finalize;
	generate_structured_answer --> quality_gate;
	init_run --> route_query;
	quality_gate -. &nbsp;pass&nbsp; .-> finalize;
	quality_gate -. &nbsp;retry&nbsp; .-> route_query;
	quality_gate -. &nbsp;stop&nbsp; .-> safe_stop;
	rerank_results --> generate_structured_answer;
	retrieve_parallel --> rerank_results;
	rewrite_query --> retrieve_parallel;
	route_query -. &nbsp;needs_clarification&nbsp; .-> ask_clarification;
	route_query -. &nbsp;answerab

In [7]:
question = 'I am on I-81 near Hagerstown with a short detour. What geology should I look for and how do I do this safely?'
result = run_stategraph(
    question,
    provider=provider,
    config={'artifact_root': 'out/stategraph/notebook_runs', 'max_retrieval_attempts': 3, 'citation_coverage_threshold': 0.80},
)
final_output = result['final_output']
print('Provider:', final_output['provider'])
print('Route:', final_output['route_decision']['decision'])
print('Quality passed:', final_output['quality']['passed'])
print('Attempts:', final_output['retrieval_attempts'])
print('Artifact dir:', result['artifact_dir'])
print('Response:\n', final_output['answer']['response'])


Provider: ollama
Route: needs_retrieval_retry
Quality passed: False
Attempts: 3
Artifact dir: out/stategraph/notebook_runs/ollama/run_20260206T002826_777b33be
Response:
 Safe stop: quality gates did not pass after maximum retrieval retries. No final grounded answer was emitted.


In [8]:
report = run_i81_eval_harness(provider='ollama', config={'artifact_root': 'out/stategraph/notebook_eval', 'max_retrieval_attempts': 3, 'citation_coverage_threshold': 0.80})
print(report['summary'])
print(report['artifact_root'])


{'provider': 'ollama', 'eval_id': 'eval_20260206T002826Z', 'question_count': 20, 'avg_latency_ms': 14.05, 'median_latency_ms': 13.76, 'citation_validity_pct': 100.0, 'pass_rate_pct': 0.0, 'pass_count': 0, 'fail_count': 20}
/Users/ryan/Developer/naturalist-companion/notebooks/out/stategraph/notebook_eval/ollama/eval_runs/eval_20260206T002826Z
